In [1]:
#Make necessary imports

import pandas as pd
import numpy as np
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
import FeatureEngineering as fe

/Users/kamielfokkink/opt/anaconda3/envs/Databases/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#Load in data

train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test-full.csv")

In [13]:
fe_train_data = fe.transformSoilType(train_data)
fe_test_data = fe.transformSoilType(test_data)

In [3]:
pca_train = fe.PCAtransform(train_data, 40)
pca_test = fe.PCAtransform(test_data, 40)

In [4]:
#classifier = RandomForestClassifier()
classifier = XGBClassifier()
#classifier = HistGradientBoostingClassifier()

pipe = make_pipeline(
    classifier
)

In [5]:
pipe.fit(pca_train, train_data['Cover_Type'])
#pipe.fit(fe_train_data.drop(columns=['Cover_Type']), fe_train_data['Cover_Type'])

/Users/kamielfokkink/opt/anaconda3/envs/Databases/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/kamielfokkink/opt/anaconda3/envs/Databases/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[19:56:24] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, enable_categorical=False,
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=8, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=0, reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=None, subsample=1,
                               tree_method='exact', validate_parameters=1,
                               verbo

In [6]:
def predict(pipe, data, outfile):
    ids = data['Id']
    prediction = pipe.predict(data)
    output = pd.DataFrame(prediction, index=ids, columns=['Cover_Type'])
    output.to_csv("submissions/" + outfile + ".csv", index=True)

In [8]:
predict(pipe, pca_test, 'XGBClassifier3')